In [10]:
import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import gzip
import re

# Hong's job description

Attached please find the venn diagrams we generated based on the lists your provided, I also sent you the shared enhanced binding gene lists for both FC>1.2 (333 genes) and FC>1.5 (87 genes). Please generate TSS centered heatmaps for these two list genes respectively. Also include Y78A in these heatmaps. The dark blue color we usually use for Flag ChIP peaks would be good.

At the meantime, you can help to generate venn diagram for WT, T1, T2 and T3, and the heatmaps for co-up peaks.

For the motif analysis, please analysis the following four groups of genes: (1) 333 genes with FC>1.2 enhanced binding; (2) 87 genes with FC>1.5 enhanced binding; (3) T3 vs.WT FC>1.5 enhanced binding. For this three, please find enriched motifs in TSS +/- 2kb region. And then (4) look for enriched motifs in wild type ENL binding peaks.

# 1. Heatmap for subgroup genes

get the TSS TES gene lists

reffile = '/mount/weili3/xc3/genomes/hg19.refGene.txt'

convert the reffile to the tss tes gene lists using awk
```shell
awk 'BEGIN{OFS="\t"}{print $3, $5, $6, $13, ".", $4}' /mount/weili3/xc3/genomes/hg19.refGene.txt > hg19.tss.tes.txt
```
remove the genes which are not on the chr1 - chr22 chrX chrY
```shell
awk '{if ($1 !~ /[_]/) print $0}' hg19.tss.tes.txt > hg19.tss.tes.trim.txt
```

## 1.1 Generate the bed file for the coup genes

In [11]:
!pwd
os.chdir('/home/xc3/experiment/ENL2/Chip-seq_analysis/no_spike_in/re_do_mapping_with_lambda/res_heatmap/')

/home/xc3/experiment/ENL2/Chip-seq_analysis/no_spike_in/re_do_mapping_with_lambda/res_heatmap_coup_peaks


In [3]:
df_genes = pd.read_csv('hg19.tss.tes.trim.txt', sep='\t', 
                       header=None, names=['chr', 'tss','tes','gene','dot','strand'],
                      dtype={1:np.int64,2:np.int64})
# drop duplicates
df_genes = df_genes.drop_duplicates(subset=['gene'])
df_genes.index = df_genes.loc[:,'gene']
df_genes.head()

,chr,tss,tes,gene,dot,strand
gene,,,,,,
NOX4,chr11,89057521,89223909,NOX4,.,-
C14orf166,chr14,52456227,52471420,C14orf166,.,+
AK3,chr9,4709556,4741309,AK3,.,-
MCM5,chr22,35796115,35820495,MCM5,.,+
FAXDC2,chr5,154198051,154230213,FAXDC2,.,-


In [86]:
df_genes.loc['WRAP73']

chr          chr1
tss       3547330
tes       3566671
gene       WRAP73
dot             .
strand          -
Name: WRAP73, dtype: object

In [93]:
gene_list = gene_lists[0]
with open(gene_list, 'r') as f:
    genes = f.readlines()
    genes = [gene.rstrip() for gene in genes]
df_sub = df_genes.reindex(genes)
df_sub[df_sub.isnull().any(axis=1)]

,chr,tss,tes,gene,dot,strand
gene,,,,,,
6-Mar,NaN,NaN,NaN,NaN,NaN,NaN


In [94]:
# generate the coup genes with tss tes
gene_lists = ['T1_2_3 co-UP FC1.2.csv', 'T1_2_3 co-UP FC1.5.csv']
names = ['1_2', '1_5']
gene_list = gene_lists[0]
for i, gene_list in enumerate(gene_lists):
    with open(gene_list, 'r') as f:
        genes = f.readlines()
        genes = [gene.rstrip() for gene in genes]
    df_genes.reindex(genes).to_csv(f'coup_{names[i]}.bed', sep='\t', header=None, index=False)

### change all the 'strand' to '+'

In [138]:
# for name in names:
#     line = 'awk \'BEGIN{OFS=\"\\t\"}{print $1,$2,$3,$4,$5,\"+\"}\'' + f' coup_{name}.bed > coup_{name}.plus.bed'
#     print(line)
#     os.system(line)

awk 'BEGIN{OFS="\t"}{print $1,$2,$3,$4,$5,"+"}' coup_1_2.bed > coup_1_2.plus.bed
awk 'BEGIN{OFS="\t"}{print $1,$2,$3,$4,$5,"+"}' coup_1_5.bed > coup_1_5.plus.bed


## 1.2 extract the signal and plot the heatmap

In [141]:
# computing the tss signal
bwpath = '/home/xc3/output/ENL2/ChIP/bw_nsp/'
exp = '293'
bwfiles = glob.glob(bwpath+exp+'*.bw')
bwfiles.sort()
bwfiles = bwfiles[:5]
for name in names:
    profile_bed = f'coup_{name}.bed'
    line = '~/software/deeptools/bin/computeMatrix reference-point --referencePoint TSS -S {} -R {} --beforeRegionStartLength 5000 --afterRegionStartLength 5000 --binSize 100 --skipZeros --missingDataAsZero -o {}.Ts.ENL.tss.gz -p 8 --sortUsingSamples 1 --outFileSortedRegions {}.sorted.bed'.format(' '.join(bwfiles), profile_bed, '{}'.format(name), name)
    print(line)
    os.system(line)


~/software/deeptools/bin/computeMatrix reference-point --referencePoint TSS -S /home/xc3/output/ENL2/ChIP/bw_nsp/293_F_ENL_treat.nsp.bw /home/xc3/output/ENL2/ChIP/bw_nsp/293_F_T1_treat.nsp.bw /home/xc3/output/ENL2/ChIP/bw_nsp/293_F_T2_treat.nsp.bw /home/xc3/output/ENL2/ChIP/bw_nsp/293_F_T3_treat.nsp.bw /home/xc3/output/ENL2/ChIP/bw_nsp/293_F_Y78A_treat.nsp.bw -R coup_1_2.bed --beforeRegionStartLength 5000 --afterRegionStartLength 5000 --binSize 100 --skipZeros --missingDataAsZero -o 1_2.Ts.ENL.tss.gz -p 8 --sortUsingSamples 1 --outFileSortedRegions 1_2.sorted.bed
~/software/deeptools/bin/computeMatrix reference-point --referencePoint TSS -S /home/xc3/output/ENL2/ChIP/bw_nsp/293_F_ENL_treat.nsp.bw /home/xc3/output/ENL2/ChIP/bw_nsp/293_F_T1_treat.nsp.bw /home/xc3/output/ENL2/ChIP/bw_nsp/293_F_T2_treat.nsp.bw /home/xc3/output/ENL2/ChIP/bw_nsp/293_F_T3_treat.nsp.bw /home/xc3/output/ENL2/ChIP/bw_nsp/293_F_Y78A_treat.nsp.bw -R coup_1_5.bed --beforeRegionStartLength 5000 --afterRegionStartLen

In [142]:
# plot the heatmap
for name in names:
    line = 'plotHeatmap --sortUsingSamples 1 -m {}.Ts.ENL.tss.gz -out {}.tss.pdf --outFileNameMatrix {} --colorMap Blues Blues Blues Blues Blues --dpi 300 --heatmapHeight 10'.format(name, name, name+'.heatmap.tss.mat')
    print(line)
    os.system(line)

plotHeatmap --sortUsingSamples 1 -m 1_2.Ts.ENL.tss.gz -out 1_2.tss.pdf --outFileNameMatrix 1_2.heatmap.tss.mat --colorMap Blues Blues Blues Blues Blues --dpi 300 --heatmapHeight 10
plotHeatmap --sortUsingSamples 1 -m 1_5.Ts.ENL.tss.gz -out 1_5.tss.pdf --outFileNameMatrix 1_5.heatmap.tss.mat --colorMap Blues Blues Blues Blues Blues --dpi 300 --heatmapHeight 10


## Make the FC 1.5 heatmap shorter

In [144]:
plotHeatmap --sortUsingSamples 1 -m 1_5.Ts.ENL.tss.gz -out 1_5.tss.pdf --outFileNameMatrix 1_5.heatmap.tss.mat --colorMap Blues Blues Blues Blues Blues --dpi 300 --heatmapHeight 5

SyntaxError: invalid syntax (<ipython-input-144-ed11eccf3690>, line 1)

## generate the signal table for the heatmap

In [67]:
def convert_deeptools_gz_to_df(file_name):
    """
    file_name: the deeptools computeMatrix output matrix gz file
    caculate the avg signal per sample across all the bins
    """
    with gzip.open(file_name, 'rt', encoding='utf-8') as f:
        lines = f.readlines()
    line = lines[0]
    a = line.split('@')[-1].split('\\')[0]
    a = re.sub('true', 'True', a)
    a = re.sub('false', 'False', a)
    a = re.sub('null', 'False', a)    
    info = eval(a)
    with open(f'{file_name}.txt', 'w') as f:
        f.writelines(lines[1:])

    df = pd.read_csv(f'{file_name}.txt', sep='\t', header=None) 
    for i, sample in enumerate(info['sample_labels']):
        start = info['sample_boundaries'][i] + 6
        end = info['sample_boundaries'][i+1] + 6    
        df[sample] = df.iloc[:, start:end].mean(axis=1)

    df.sort_values(by=[info['sample_labels'][0]], inplace=True, ascending=False)     
    return df

In [75]:
for name in names:
    df = convert_deeptools_gz_to_df(file_name = f'{name}.Ts.ENL.tss.gz')
    df.iloc[:, [0,1,2,3,4,5,-5,-4,-3,-2,-1]].to_csv(f'{name}.tss.sig.txt', index=False, sep='\t')    

# 2. Generate the venn diagram for peaks for T1 T2 T3

In [101]:
workPath = '/home/xc3/experiment/ENL2/Chip-seq_analysis/no_spike_in/re_do_mapping_with_lambda/res_venn_diagram/'
os.chdir(workPath)
!pwd

/home/xc3/experiment/ENL2/Chip-seq_analysis/no_spike_in/re_do_mapping_with_lambda/res_venn_diagram


In [124]:
exp = '293'
bed_path = '/mount/weili3/xc3/ENL2_ChIP/res_nsp_lambda/'
bedFiles = glob.glob(f'{bed_path}{exp}*nsp_peaks.bed')
# extract the first three columns of the bed files
for bedFile in bedFiles:
    outFile = bedFile.split('/')[-1].split('.')[0]
    line = "awk \'BEGIN{OFS=\"\\t\"}{print $1,$2,$3}\'" + f' {bedFile} > {outFile}.bed'
    print(line)
    os.system(line)

awk 'BEGIN{OFS="\t"}{print $1,$2,$3}' /mount/weili3/xc3/ENL2_ChIP/res_nsp_lambda/293_F_Y78A.nsp_peaks.bed > 293_F_Y78A.bed
awk 'BEGIN{OFS="\t"}{print $1,$2,$3}' /mount/weili3/xc3/ENL2_ChIP/res_nsp_lambda/293_F_T2.nsp_peaks.bed > 293_F_T2.bed
awk 'BEGIN{OFS="\t"}{print $1,$2,$3}' /mount/weili3/xc3/ENL2_ChIP/res_nsp_lambda/293_F_ENL.nsp_peaks.bed > 293_F_ENL.bed
awk 'BEGIN{OFS="\t"}{print $1,$2,$3}' /mount/weili3/xc3/ENL2_ChIP/res_nsp_lambda/293_F_T1.nsp_peaks.bed > 293_F_T1.bed
awk 'BEGIN{OFS="\t"}{print $1,$2,$3}' /mount/weili3/xc3/ENL2_ChIP/res_nsp_lambda/293_F_T3.nsp_peaks.bed > 293_F_T3.bed


In [113]:
nodes_ppn = [1, 1]
memory = 4
wait_time = [20, 00]
quene = "short"
files = glob.glob('*.bed')
files.sort()
files = files[:-1]
names = [ifile.split('/')[-1].split('.')[0] for ifile in files]

In [122]:
# open the initialization header for pbs and read the contents
with open('/home/xc3/experiment/initial.pbs','r') as f:
    pbs_header = f.readlines()

# write each sample a pbs files
experiment_name = "{}.ato".format('venn_diagram')
with open(experiment_name,'w') as f:
    pbs_initial = pbs_header[:]
    # configuration for the experiments
    # pbs_initial[1]::job name
    pbs_initial[1] = pbs_initial[1].format('venn')
    # pbs_initial[2]::nodes and ppn
    pbs_initial[2] = pbs_initial[2].format(*nodes_ppn)
    # pbs_initial[4]::memeroy
    pbs_initial[4] = pbs_initial[4].format(memory)
    # pbs_initial[5]::waiting time
    pbs_initial[5] = pbs_initial[5].format(*wait_time)
    # pbs_initial[11]::err output
    pbs_initial[11] = pbs_initial[11].format('venn')
    # pbs_initial[12]::log
    pbs_initial[12] = pbs_initial[12].format('venn')
    # pbs_initial[14]::quene
    pbs_initial[14] = pbs_initial[14].format(quene)
    
    #write all the configurations into the pbs file
    for line in pbs_initial:
        f.write(line)
    line = f'cd {workPath}'
    f.write(line + '\n')

    treat_files = files
    print(treat_files)
    treat_files_format = '{} ' * len(treat_files)
    treat_files_format = treat_files_format.format(*treat_files)
    output_venn = '{}_all.venn.pdf'.format(exp)
    output_bed = '{}_all.venn.bed'.format(exp)
    line = 'python /home/xc3/software/bed_overlap4.py -v {} -i {} {}'.format(output_venn, output_bed, treat_files_format)
    print(line)
    f.write(line + '\n')    

['293_F_ENL.bed', '293_F_T1.bed', '293_F_T2.bed', '293_F_T3.bed']
python /home/xc3/software/bed_overlap4.py -v 293_all.venn.pdf -i 293_all.venn.bed 293_F_ENL.bed 293_F_T1.bed 293_F_T2.bed 293_F_T3.bed 


# 3. Heatmap for subgroup coup peaks

The coup peaks comes from the venn analysis intersection from bedoverlap

In [12]:
workPath = '/home/xc3/experiment/ENL2/Chip-seq_analysis/no_spike_in/re_do_mapping_with_lambda/res_heatmap_coup_peaks/'
os.chdir(workPath)
!pwd

/home/xc3/experiment/ENL2/Chip-seq_analysis/no_spike_in/re_do_mapping_with_lambda/res_heatmap_coup_peaks


## 3.1 extract the 1.2 / 1.5 peaks as bed files then use bedoverlap to get the intersect

In [7]:
ths = [1.2, 1.5]
names = ['1_2','1_5']

In [207]:
peakFiles = glob.glob('*.pval.txt')
peakFiles.sort()
for th, name in zip(ths, names):
    for peakFile in peakFiles:
        peakDf = pd.read_csv(peakFile, sep='\t', header=None, skiprows=[0])
        subId = peakDf.iloc[:,4].div(peakDf.iloc[:,5]) >= th
        print(peakDf[subId].shape,'-->', peakFile, f'{th}')
        outFile = peakFile.split(".")[0] + f'.{name}.bed'
        print(outFile)
        peakDf[subId].iloc[:, [0,1,2]].to_csv(f'{outFile}', sep='\t', header=None, index=False)

(409, 12) --> 293_T1_WT.merge.counts.anno.pval.txt 1.2
293_T1_WT.1_2.bed
(2287, 12) --> 293_T2_WT.merge.counts.anno.pval.txt 1.2
293_T2_WT.1_2.bed
(3761, 12) --> 293_T3_WT.merge.counts.anno.pval.txt 1.2
293_T3_WT.1_2.bed
(109, 12) --> 293_T1_WT.merge.counts.anno.pval.txt 1.5
293_T1_WT.1_5.bed
(495, 12) --> 293_T2_WT.merge.counts.anno.pval.txt 1.5
293_T2_WT.1_5.bed
(1107, 12) --> 293_T3_WT.merge.counts.anno.pval.txt 1.5
293_T3_WT.1_5.bed


## 3.2 find the T1 T2 T3 overlapped peaks


In [212]:
for name in names:
    bedFiles = glob.glob(f'*{name}.bed')
    outBed = f'293.all.{name}.bed'
    outVenn = f'293.all.{name}.venn.pdf'
    bed_files_format = ' '.join(bedFiles)
    line = 'python /home/xc3/software/bed_overlap3.py -v {} -i {} {}'.format(outVenn, outBed, bed_files_format)
    print(line)
    

python /home/xc3/software/bed_overlap3.py -v 293.all.1_2.venn.pdf -i 293.all.1_2.bed 293_T1_WT.1_2.bed 293_T2_WT.1_2.bed 293_T3_WT.1_2.bed
python /home/xc3/software/bed_overlap3.py -v 293.all.1_5.venn.pdf -i 293.all.1_5.bed 293_T1_WT.1_5.bed 293_T2_WT.1_5.bed 293_T3_WT.1_5.bed


## 3.3 Redraw the venn diagram for the shared 1.2 1.5 peaks
the results depends hong whether like it or not (TODO)

## 3.4 Extract the signal for heatmap
using the previous above 

In [223]:
#format the bed file into the bed file add three columns more; 
for name in names:
    Bed = f'293.all.{name}.bed'
    outBed = f'293.all.{name}.heat.bed'
    line = "awk 'BEGIN{OFS=\"\\t\"}{if (NR>1){print $1,$2,$3,\"none\",\".\",\"+\"}}'" + f' {Bed} > {outBed}'
    print(line)
    os.system(line)

awk 'BEGIN{OFS="\t"}{if (NR>1){print $1,$2,$3,"none",".","+"}}' 293.all.1_2.bed > 293.all.1_2.heat.bed
awk 'BEGIN{OFS="\t"}{if (NR>1){print $1,$2,$3,"none",".","+"}}' 293.all.1_5.bed > 293.all.1_5.heat.bed


In [222]:
!ls

293.all.1_2.bed			      293_T2_WT.1_2.bed
293.all.1_2.heat.bed		      293_T2_WT.1_5.bed
293.all.1_2.venn.pdf		      293_T2_WT.merge.counts.anno.pval.txt
293.all.1_5.bed			      293_T2_WT.merge.counts.anno.txt
293.all.1_5.heat.bed		      293_T3_WT.1_2.bed
293.all.1_5.venn.pdf		      293_T3_WT.1_5.bed
293_T1_WT.1_2.bed		      293_T3_WT.merge.counts.anno.pval.txt
293_T1_WT.1_5.bed		      293_T3_WT.merge.counts.anno.txt
293_T1_WT.merge.counts.anno.pval.txt  293_all.venn.bed
293_T1_WT.merge.counts.anno.txt


bin size = 100

In [224]:
bwpath = '/home/xc3/output/ENL2/ChIP/bw_nsp/'
exp = '293'
bwfiles = glob.glob(bwpath+exp+'*.bw')
bwfiles.sort()
bwfiles = bwfiles[:5]
for name in names:
    outBed = f'293.all.{name}.heat.bed'
    line = '~/software/deeptools/bin/computeMatrix reference-point --referencePoint center -S {} -R {} --beforeRegionStartLength 5000 --afterRegionStartLength 5000 --binSize 100 --skipZeros --missingDataAsZero -o {}.Ts.ENL.peak.gz -p 8'.format(' '.join(bwfiles), outBed, '{}'.format(name))
    print(line)
    os.system(line)

~/software/deeptools/bin/computeMatrix reference-point --referencePoint center -S /home/xc3/output/ENL2/ChIP/bw_nsp/293_F_ENL_treat.nsp.bw /home/xc3/output/ENL2/ChIP/bw_nsp/293_F_T1_treat.nsp.bw /home/xc3/output/ENL2/ChIP/bw_nsp/293_F_T2_treat.nsp.bw /home/xc3/output/ENL2/ChIP/bw_nsp/293_F_T3_treat.nsp.bw /home/xc3/output/ENL2/ChIP/bw_nsp/293_F_Y78A_treat.nsp.bw -R 293.all.1_2.heat.bed --beforeRegionStartLength 5000 --afterRegionStartLength 5000 --binSize 100 --skipZeros --missingDataAsZero -o 1_2.Ts.ENL.peak.gz -p 8
~/software/deeptools/bin/computeMatrix reference-point --referencePoint center -S /home/xc3/output/ENL2/ChIP/bw_nsp/293_F_ENL_treat.nsp.bw /home/xc3/output/ENL2/ChIP/bw_nsp/293_F_T1_treat.nsp.bw /home/xc3/output/ENL2/ChIP/bw_nsp/293_F_T2_treat.nsp.bw /home/xc3/output/ENL2/ChIP/bw_nsp/293_F_T3_treat.nsp.bw /home/xc3/output/ENL2/ChIP/bw_nsp/293_F_Y78A_treat.nsp.bw -R 293.all.1_5.heat.bed --beforeRegionStartLength 5000 --afterRegionStartLength 5000 --binSize 100 --skipZeros -

In [8]:
# plot the heatmap
for name in names:
    line = 'plotHeatmap --sortUsingSamples 1 -m {}.Ts.ENL.peak.gz -out {}.peak.pdf --outFileNameMatrix {} --colorMap Blues Blues Blues Blues Blues --dpi 300 --heatmapHeight 10 --refPointLabel center'.format(name, name, name+'.heatmap.peak.mat')
    print(line)
    os.system(line)

plotHeatmap --sortUsingSamples 1 -m 1_2.Ts.ENL.peak.gz -out 1_2.peak.pdf --outFileNameMatrix 1_2.heatmap.peak.mat --colorMap Blues Blues Blues Blues Blues --dpi 300 --heatmapHeight 10 --refPointLabel center
plotHeatmap --sortUsingSamples 1 -m 1_5.Ts.ENL.peak.gz -out 1_5.peak.pdf --outFileNameMatrix 1_5.heatmap.peak.mat --colorMap Blues Blues Blues Blues Blues --dpi 300 --heatmapHeight 10 --refPointLabel center


## replot the heatmap with shorter height

In [ ]:
plotHeatmap --sortUsingSamples 1 -m 1_5.Ts.ENL.peak.gz -out 1_5.peak.pdf --outFileNameMatrix 1_5.heatmap.peak.mat --colorMap Blues Blues Blues Blues Blues --dpi 300 --heatmapHeight 5 --refPointLabel center

## Generate the table for the peak heatmap using peak center

In [79]:
for name in names:
    df = convert_deeptools_gz_to_df(file_name = f'{name}.Ts.ENL.peak.gz')
    df.iloc[:, [0,1,2,3,4,5,-5,-4,-3,-2,-1]].to_csv(f'{name}.peak.sig.txt', index=False, sep='\t') 

## add the delta heatmap for the above sampels
Just heard back from Liling, actually she wants to change the delta heatmap color theme back to grey/black and orange that we used before. Could you please make this change for all the delta heatmap? In the FC >1.2 and FC>1.5 delta heatmap, right now we only have signals for wt, T1, T2 and T3, please add T1-wt, T2-wt and T3-wt. Attached is an old version of this figure for your reference, keep the blue color for individual sample, use the same organge-black/grey for delta map, but no need to label gene names on the right. **2018-8-3**

In [9]:
!ls *.gz
!ls *.pdf

1_2.Ts.ENL.peak.gz  1_5.Ts.ENL.peak.gz
1_2.peak.pdf   1_5.peak.pdf   293.all.1_2.venn.pdf
1_2.peak1.pdf  1_5.peak1.pdf  293.all.1_5.venn.pdf


## Generate the table for the peak region

In [107]:
# function for formating the chr start end into bed format
# if the last column has sign ('+' or '-'), then prepare to swap tss and tes for '-'
def convert_region_to_bed(bedfile, outname, header=None):
    df = pd.read_csv(bedfile,sep='\t',header=None)
    df.columns = np.arange(0,df.shape[1])
    # detect the strand and swap the start end when negative strand
    # the last column should be strand
    if df.iloc[0,-1] == '-' or df.iloc[0,-1] == '+':
        idx = df.iloc[:,-1] == '-'
        # swap the tss tes according to the strand
        df.loc[idx,[1,2]] = df.loc[idx,[2,1]].values
        # modify the strand
        df.iloc[:,-1] = '+'
        
    df.loc[:,3] = np.arange(df.shape[0])# for gene name column
    df.loc[:,4] = 0 # for bed format (value)
    df.loc[:,5] = '+'
    df = df.reindex(range(6), axis='columns')
    print('writing the output --> {}'.format(outname))
    print(f'{outname} --->: reformated bed_file as the input of the bigWigAverageoverBed')
    df.to_csv(outname,sep='\t',index=False,header=None)
    
    
def comp_avg_sig_from_bw_by_bed(bedfile, bw_files, which_column=2):
    """
    bedfile: the input merged bedfile which needs to extract the signal from
    bw_files: list of bw files whose signal is extracted from
    which_column: 3 is the acutal counts column, 4 is the acutal avg signal column;
    but, we need to use the first column as index, then 2 is the counts column;
    outname: reformat the bedfile to the normal 6 columns bed file
    will save the output file in the same folder as the bedfile
    """
    outname = bedfile.split('.bed')[0] + '.str.bed'
    # convert the 3 columns bed to the normal bed file
    convert_region_to_bed(bedfile, outname)
    for i,bw_file in enumerate(bw_files):
        command = line = f'~/software/bigWigAverageOverBed {bw_file} {outname} avg_sig_{i}.txt'
        print('execute --> {}'.format(command))
        os.system(command)
    dfs = []
    for i in range(len(bw_files)):
        dfs.append(pd.read_csv(f'avg_sig_{i}.txt',sep='\t',header=None, index_col=0))
    df = pd.DataFrame([dfs[0].iloc[:,0]] + # iloc[:,0]: the width
                      [dfi.iloc[:,which_column] for dfi in dfs]).T # conbine the columns together
    df.columns = ['width'] + [bw_file.split('/')[-1].split('.')[0] for bw_file in bw_files]
    df_bed = pd.read_csv(bedfile, sep='\t', header=None, names = ['chr', 'start', 'end'])
    df_out = df_bed.join(df)
    print(f"output file -->: {bedfile.split('.bed')[0] + '.counts.txt'}")
    df_out.to_csv(bedfile.split('.bed')[0] + '.counts.txt', sep='\t', index=False)

In [112]:
bwpath = '/home/xc3/output/ENL2/ChIP/bw_nsp/'
exp = '293'
bwfiles = glob.glob(bwpath+exp+'*.bw')
bwfiles.sort()
bwfiles = bwfiles[:5]
for name in names:
    bed_file = f'{exp}.all.{name}.bed'
    print('generate the signal for',bed_file)
    comp_avg_sig_from_bw_by_bed(bed_file, bwfiles, 3)
    

generate the signal for 293.all.1_2.bed
writing the output --> 293.all.1_2.str.bed
293.all.1_2.str.bed --->: reformated bed_file as the input of the bigWigAverageoverBed
execute --> ~/software/bigWigAverageOverBed /home/xc3/output/ENL2/ChIP/bw_nsp/293_F_ENL_treat.nsp.bw 293.all.1_2.str.bed avg_sig_0.txt
execute --> ~/software/bigWigAverageOverBed /home/xc3/output/ENL2/ChIP/bw_nsp/293_F_T1_treat.nsp.bw 293.all.1_2.str.bed avg_sig_1.txt
execute --> ~/software/bigWigAverageOverBed /home/xc3/output/ENL2/ChIP/bw_nsp/293_F_T2_treat.nsp.bw 293.all.1_2.str.bed avg_sig_2.txt
execute --> ~/software/bigWigAverageOverBed /home/xc3/output/ENL2/ChIP/bw_nsp/293_F_T3_treat.nsp.bw 293.all.1_2.str.bed avg_sig_3.txt
execute --> ~/software/bigWigAverageOverBed /home/xc3/output/ENL2/ChIP/bw_nsp/293_F_Y78A_treat.nsp.bw 293.all.1_2.str.bed avg_sig_4.txt
output file -->: 293.all.1_2.counts.txt
generate the signal for 293.all.1_5.bed
writing the output --> 293.all.1_5.str.bed
293.all.1_5.str.bed --->: reformat

### re-rank the file accordi ng to the heatmap

In [139]:
def combine_row(row):
    row = [str(ele) for ele in row]
    return '|'.join(row)

for name in names:
    df1 = pd.read_csv(f'{name}.peak.sig.txt', sep='\t')
    df1.index = df1.iloc[:,:3].apply(combine_row, axis=1)
    df2 = pd.read_csv(f'{exp}.all.{name}.counts.txt', sep='\t')
    df2.index = df2.iloc[:,:3].apply(combine_row, axis=1)
    df2 = df2.reindex(df1.index)
    df2.to_csv(f'{exp}.all.{name}.sig.txt', sep='\t', index=False)


In [80]:
def annoate_bed_to_gene(infile, outfile, cutoff=5000):
    ref, tss_set = {}, set()
    reffile = '/mount/weili3/xc3/genomes/hg19.refGene.txt'
    for line in open(reffile):
        col = line.split('\t')
        name, cr, strand, TSS, TES, symbol = col[1], col[2], col[3], int(col[4]), int(col[5]), col[12]
        if strand == '-': TSS, TES = TES, TSS
        if cr not in ref: ref[cr] = []
        if (cr,TSS,strand) not in tss_set:
            ref[cr].append((name,symbol,strand,TSS,TES))
            tss_set.add((cr,TSS,strand))
    for cr in ref: ref[cr].append(('none','none','none',0,0))
    
    # annotate the file
    text = open(infile).readlines()
    fout = open(outfile, 'w')
    print('processing on {}\n will output {}\n'.format(infile,outfile))
    for line in text:
        col = line.split('\t')
        try: cr, start, end = col[0], int(col[1]), int(col[2])
        except: 
            fout.write(line[:-1]+'\twithin_genebody\tnearest_TSS\tdistance\n')
            continue
        if cr not in ref: continue
        peak = (start + end) / 2
        genes, genebody, genes0, genebody0 = [], [], [], []
        for name, symbol, strand, TSS, TES in ref[cr]:
            if strand == '+':
                dist = end - TSS
                if abs(start-TSS)<abs(dist):dist=start-TSS

            elif strand == '-':
                dist = TSS - end
                if abs(TSS - start)<abs(dist):dist=TSS - start

            elif strand != 'none': raise ValueError
            if abs(dist) <= cutoff: genes.append((abs(dist),symbol,dist)) 
            if (start - TSS) * (start - TES) <= 0: genebody.append((abs(dist),symbol,dist))
            elif (end - TSS) * (end - TES) <= 0: genebody.append((abs(dist),symbol,dist))
            elif (start - TSS) * (end - TES) <=0: genebody.append((abs(dist),symbol,dist))
        genes, genebody = sorted(genes), sorted(genebody)
        for g in sorted(genes):
            if g[1] not in [x[1] for x in genes0]: genes0.append(g)
        for g in sorted(genebody):
            if g[1] not in [x[1] for x in genebody0]: genebody0.append(g)
        if any(genes0):
            symbols = ','.join([x[1] for x in genes0])
            dists = ','.join(['%d' % x[2] for x in sorted(genes0)])
        else: symbols, dists = 'none', 'none'
        if any(genebody): body = ','.join([x[1] for x in genebody0])
        else: body = 'none'
        fout.write(line[:-1] + '\t%s\t%s\t%s\n' % (body,symbols,dists))
    fout.close()   

In [140]:
for name in names:
    annoate_bed_to_gene(f'{exp}.all.{name}.sig.txt', f'{exp}.all.{name}.sig.anno.txt')

processing on 293.all.1_2.sig.txt
 will output 293.all.1_2.sig.anno.txt

processing on 293.all.1_5.sig.txt
 will output 293.all.1_5.sig.anno.txt



# 4 Motif analysis

1） 333 genes with FC>1.2 enhanced binding; 2）87 genes with FC>1.5; 3) T3 vs.WT FC>1.5 enhanced binding

In [10]:
workPath = '/home/xc3/experiment/ENL2/Chip-seq_analysis/no_spike_in/re_do_mapping_with_lambda/res_motif/'
os.chdir(workPath)
!pwd

/home/xc3/experiment/ENL2/Chip-seq_analysis/no_spike_in/re_do_mapping_with_lambda/res_motif


# 4.1 extract the sequence for the three gene lists

In [266]:
# read the gene's tss and tes and save to the bed file
geneLists = ['T1_2_3 co-UP FC1.2.txt', 'T1_2_3 co-UP FC1.5.txt', '293_T3_WT.merge.counts.anno.genes1_5.txt']
names = ['coup.1_2', 'coup.1_5', 'T3WT.1_5']
gene_list = geneLists[0]
for i, gene_list in enumerate(geneLists):
    with open(gene_list, 'r') as f:
        genes = f.readlines()
        genes = [gene.rstrip() for gene in genes]
        df = df_genes.reindex(genes)
        df.loc[:,'strand'] = '+'
        print(f'process on {gene_list}')
        df['tes'] = df['tss'] + 2000
        df['tss'] = df['tss'] - 2000
        df.to_csv(f'{names[i]}.motif.bed', sep='\t', header=None, index=False)

process on T1_2_3 co-UP FC1.2.txt
process on T1_2_3 co-UP FC1.5.txt
process on 293_T3_WT.merge.counts.anno.genes1_5.txt


In [272]:
# extract the sequence for the corrdinates
for name in names:
    bedFile = f'{name}.motif.bed'
    outTxt = f'{name}.motif.txt'
    line = f"bedtools getfasta -fi /mount/weili3/xc3/genomes/hg19_trim.fa -bed {bedFile} -fo {outTxt}"
    print(line)
    os.system(line)
    line = "awk \'!a[$0]++\'" + f" {outTxt} > {name}.motif.uniq.txt"
    print(line)
    os.system(line)

bedtools getfasta -fi /mount/weili3/xc3/genomes/hg19_trim.fa -bed coup.1_2.motif.bed -fo coup.1_2.motif.txt
awk '!a[$0]++' coup.1_2.motif.txt > coup.1_2.motif.uniq.txt
bedtools getfasta -fi /mount/weili3/xc3/genomes/hg19_trim.fa -bed coup.1_5.motif.bed -fo coup.1_5.motif.txt
awk '!a[$0]++' coup.1_5.motif.txt > coup.1_5.motif.uniq.txt
bedtools getfasta -fi /mount/weili3/xc3/genomes/hg19_trim.fa -bed T3WT.1_5.motif.bed -fo T3WT.1_5.motif.txt
awk '!a[$0]++' T3WT.1_5.motif.txt > T3WT.1_5.motif.uniq.txt


## 4.2 look for enriched motifs in wild type ENL binding peaks.

In [12]:
!head /mount/weili3/xc3/ENL2_ChIP/res_nsp_lambda/293_F_ENL.nsp_peaks.bed
!wc -l /mount/weili3/xc3/ENL2_ChIP/res_nsp_lambda/293_F_ENL.nsp_peaks.bed
!cp /mount/weili3/xc3/ENL2_ChIP/res_nsp_lambda/293_F_ENL.nsp_peaks.bed  ./
!mv 293_F_ENL.nsp_peaks.bed ENL.motif.bed

chr1	858702	861593	MACS_peak_1	1093.75
chr1	870327	871455	MACS_peak_2	97.85
chr1	875392	878559	MACS_peak_3	676.52
chr1	932422	936620	MACS_peak_4	1067.60
chr1	955250	956213	MACS_peak_5	86.76
chr1	1709153	1710245	MACS_peak_6	94.82
chr1	1821251	1822759	MACS_peak_7	346.39
chr1	2159906	2163136	MACS_peak_8	1053.44
chr1	2245753	2246888	MACS_peak_9	88.14
chr1	3568419	3569644	MACS_peak_10	98.29
3298 /mount/weili3/xc3/ENL2_ChIP/res_nsp_lambda/293_F_ENL.nsp_peaks.bed


In [13]:
name = 'ENL'
bedFile = f'{name}.motif.bed'
outTxt = f'{name}.motif.txt'
line = f"bedtools getfasta -fi /mount/weili3/xc3/genomes/hg19_trim.fa -bed {bedFile} -fo {outTxt}"
print(line)
os.system(line)
line = "awk \'!a[$0]++\'" + f" {outTxt} > {name}.motif.uniq.txt"
print(line)
os.system(line)

bedtools getfasta -fi /mount/weili3/xc3/genomes/hg19_trim.fa -bed ENL.motif.bed -fo ENL.motif.txt
awk '!a[$0]++' ENL.motif.txt > ENL.motif.uniq.txt


0

In [141]:
!pwd
os.chdir('/home/xc3/experiment/ENL2/Chip-seq_analysis/no_spike_in/re_do_mapping_with_lambda/res_heatmap_coup_peaks')

/home/xc3/experiment/ENL2/Chip-seq_analysis/no_spike_in/re_do_mapping_with_lambda/res_heatmap_coup_peaks


## 5. Test all the pre-binding peaks signal

In [143]:
bwfiles

['/home/xc3/output/ENL2/ChIP/bw_nsp/293_F_ENL_treat.nsp.bw',
 '/home/xc3/output/ENL2/ChIP/bw_nsp/293_F_T1_treat.nsp.bw',
 '/home/xc3/output/ENL2/ChIP/bw_nsp/293_F_T2_treat.nsp.bw',
 '/home/xc3/output/ENL2/ChIP/bw_nsp/293_F_T3_treat.nsp.bw',
 '/home/xc3/output/ENL2/ChIP/bw_nsp/293_F_Y78A_treat.nsp.bw']

In [149]:
treats = ['T1','T2','T3']
treat = treats[0]
ctrl = 'ENL'
[glob.glob(bwpath+exp+'*'+treat+'*.bw')[0], glob.glob(bwpath+exp+'*'+ctrl+'*.bw')[0]]
bed_file = f'{exp}_{treat}_WT.1_2.bed'
!head $bed_file

chr1	8482987	8484389
chr1	15943726	15944596
chr1	16173761	16176666
chr1	26734734	26735700
chr1	27286358	27287134
chr1	38229600	38230601
chr1	57043618	57045418
chr1	61547212	61550060
chr1	65431315	65432543
chr1	78097423	78099183


In [171]:
bwpath = '/home/xc3/output/ENL2/ChIP/bw_nsp/'
exp = '293'
bwfiles.sort()
for treat in treats:
    bed_file = f'{exp}_{treat}_WT.1_2.bed'
    bwfiles = [glob.glob(bwpath+exp+'*'+treat+'*.bw')[0],
          glob.glob(bwpath+exp+'*'+ctrl+'*.bw')[0]]
    print('generate the signal for',bed_file)
    comp_avg_sig_from_bw_by_bed(bed_file, bwfiles, 3)
    print('********'+'\n')

generate the signal for 293_T1_WT.1_2.bed
writing the output --> 293_T1_WT.1_2.str.bed
293_T1_WT.1_2.str.bed --->: reformated bed_file as the input of the bigWigAverageoverBed
execute --> ~/software/bigWigAverageOverBed /home/xc3/output/ENL2/ChIP/bw_nsp/293_F_T1_treat.nsp.bw 293_T1_WT.1_2.str.bed avg_sig_0.txt
execute --> ~/software/bigWigAverageOverBed /home/xc3/output/ENL2/ChIP/bw_nsp/293_F_ENL_treat.nsp.bw 293_T1_WT.1_2.str.bed avg_sig_1.txt
output file -->: 293_T1_WT.1_2.counts.txt
********
generate the signal for 293_T2_WT.1_2.bed
writing the output --> 293_T2_WT.1_2.str.bed
293_T2_WT.1_2.str.bed --->: reformated bed_file as the input of the bigWigAverageoverBed
execute --> ~/software/bigWigAverageOverBed /home/xc3/output/ENL2/ChIP/bw_nsp/293_F_T2_treat.nsp.bw 293_T2_WT.1_2.str.bed avg_sig_0.txt
execute --> ~/software/bigWigAverageOverBed /home/xc3/output/ENL2/ChIP/bw_nsp/293_F_ENL_treat.nsp.bw 293_T2_WT.1_2.str.bed avg_sig_1.txt
output file -->: 293_T2_WT.1_2.counts.txt
********


In [185]:
# df = pd.read_csv('1_2.peak.sig.txt', sep='\t')
df = pd.read_csv('293_T1_WT.1_2.counts.txt', sep='\t')

In [186]:
df['fc'] = df.iloc[:,-2].div(df.iloc[:,-1])

In [187]:
df[df['fc'] < 1.2]

,chr,start,end,width,293_F_T1_treat,293_F_ENL_treat,fc
